In [2]:
from urllib.request import urlopen
import pandas as pd
import os
HOME = os.environ['HOME']

In [3]:
labelfname = os.path.join(HOME, '/Users/klay6683/Downloads/index.lbl.txt')
tabfname = os.path.join(HOME, '/Users/klay6683/Downloads/index.tab.txt')

In [66]:
pd.read_csv(tabfname, skipinitialspace=True, header=None, quotechar='"',
            quoting=2).shape

(7227, 139)

In [69]:
df.irow(0)

0                                 W1656702236_1.IMG     
1       data/1656702236_1656971527/W1656702236_1.IMG    
2                                            COISS_2064 
3                                                   OFF 
4                                                 72.576
5                                                   OFF 
6      trigger_30737_2.ioi                           ...
7                                                  30737
8                                               72.32388
9                                                  12BIT
10              CO-S-ISSNA/ISSWA-2-EDR-V1.0             
11                                                  NO  
12     N/A                                           ...
13                                             -87.89516
14                                 2010-183T17:00:41.255
15                                 2010-183T17:01:29.175
16                                                   112
17                             

In [72]:
with open(tabfname) as f:
    line = f.readline()

In [75]:
line = line.split(',')

In [79]:
line =[i.strip('"').strip() for i in line]

In [84]:
dfline = df.irow(0).values.tolist()

In [87]:
dfline = [str(i).strip() for i in dfline]

In [90]:
len(line)

152

In [103]:
for i in range(len(dfline)):
    if dfline[i] != line[i]:
        try:
            if int(float(dfline[i])) != int(float(line[i])):
                print(i)
                break
        except:
            print(i)
            break

117


In [104]:
dfline[i]

'naif0009.tls,cpck14Dec2001.tpc,cpck_rock_21Jan2011_merged.tpc,cpck09May2011.tpc,sat164.bsp,100913R_SCPSE_10178_10216.bsp,110317AP_RE_90165_18018.bsp,10176_10211pg_live.bc,10179_10184ra.bc,cpck20May2011.tpc,cas_v40.tf,cas_iss_v10.ti,cas00148.tsc'

In [105]:
line[i]

'naif0009.tls'

In [107]:
pd.read_csv?

In [99]:
int(float('-9.0'))

-9

In [92]:
dfline[51]

'-999.0'

In [93]:
line[51]

'-999'

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7227 entries, 0 to 7226
Columns: 139 entries, 0 to 138
dtypes: float64(68), int64(21), object(50)
memory usage: 7.7+ MB


# Parsing label file

In [6]:
import pvl

In [12]:
label = pvl.load(labelfname)
label.keys()

dict_keys(['IMAGE_INDEX_TABLE', 'FILE_RECORDS', '^IMAGE_INDEX_TABLE', 'PDS_VERSION_ID', 'RECORD_TYPE', 'RECORD_BYTES'])

In [20]:
index_table = label['IMAGE_INDEX_TABLE']

In [33]:
index_table.keys()

dict_keys(['INDEX_TYPE', 'INTERCHANGE_FORMAT', 'COLUMN', 'COLUMNS', 'ROW_BYTES', 'ROWS'])

In [67]:
index_table['COLUMNS']

118

In [109]:
columns = index_table.getlist('COLUMN')

In [110]:
colnames = [i['NAME'] for i in columns]

In [115]:
from itertools import zip_longest

In [121]:
i = 0
for col, lineitem, dfitem in zip_longest(colnames, line, dfline):
    print(col, i)
    print(lineitem)
    print(dfitem)
    print()
    i += 1

FILE_NAME 0
W1656702236_1.IMG
W1656702236_1.IMG

FILE_SPECIFICATION_NAME 1
data/1656702236_1656971527/W1656702236_1.IMG
data/1656702236_1656971527/W1656702236_1.IMG

VOLUME_ID 2
COISS_2064
COISS_2064

ANTIBLOOMING_STATE_FLAG 3
OFF
OFF

BIAS_STRIP_MEAN 4
72.575996
72.575996

CALIBRATION_LAMP_STATE_FLAG 5
OFF
OFF

COMMAND_FILE_NAME 6
trigger_30737_2.ioi
trigger_30737_2.ioi

COMMAND_SEQUENCE_NUMBER 7
30737
30737

DARK_STRIP_MEAN 8
72.323875
72.323875

DATA_CONVERSION_TYPE 9
12BIT
12BIT

DATA_SET_ID 10
CO-S-ISSNA/ISSWA-2-EDR-V1.0
CO-S-ISSNA/ISSWA-2-EDR-V1.0

DELAYED_READOUT_FLAG 11
NO
NO

DESCRIPTION 12
N/A
N/A

DETECTOR_TEMPERATURE 13
-87.895164
-87.895164

EARTH_RECEIVED_START_TIME 14
2010-183T17:00:41.255
2010-183T17:00:41.255

EARTH_RECEIVED_STOP_TIME 15
2010-183T17:01:29.175
2010-183T17:01:29.175

ELECTRONICS_BIAS 16
112
112

EXPECTED_MAXIMUM 17
79.504997
79.504997

EXPECTED_PACKETS 18
87.655296
87.655296

EXPOSURE_DURATION 19
585
585

FILTER_NAME 20
320
320

FILTER_TEMPERATURE 21
CL1

In [58]:
df.columns = columns

ValueError: Length mismatch: Expected axis has 139 elements, new values have 118 elements

In [123]:
def tokenize(line):
    tokens = line.split('=')
    return [i.strip() for i in tokens]

In [124]:
class PDSColumn:
    def __init__(self, buffer):
        d = {}
        i = 0
        for line in buffer:
            if i > 10:
                break
            tokens = tokenize(line)
            if tokens[0] == 'OBJECT':
                continue
            elif tokens[0] == 'END_OBJECT':
                break
            try:
                d[tokens[0].strip()] = tokens[1].strip()
            except IndexError:
                continue
            i += 1
        self.d = d

In [125]:
class PDSLabelReader:
    def __init__(self, fname):
        f = open(fname)
        # pre-parse the stuff away we don't need
        while True:
            tokens = tokenize(f.readline())
            try:
                if tokens[1] == 'COLUMN':
                    break
            except IndexError:
                continue
        self.f = f
        self.parse_columns()
        self.expand_columns()
    def parse_columns(self):
        columns = []
        while True:
            col = PDSColumn(self.f)
            if len(col.d) != 0:
                columns.append(col)
            else:
                break
        self.columns = columns
    def expand_columns(self):
        final_cols = []
        for pdscol in self.columns:
            dic = pdscol.d
            if 'ITEMS' in dic:
                no_items = int(dic['ITEMS'])
                for i in range(no_items):
                    final_cols.append("{}_{}".format(dic['NAME'],i))
            else:
                final_cols.append(dic['NAME'])
        self.final_cols = final_cols
labelfile = PDSLabelReader(labelfname)
df.columns = labelfile.final_cols

ValueError: Length mismatch: Expected axis has 139 elements, new values have 52 elements

In [132]:
cleaned_colnames = [i for i in labelfile.final_cols]
cleaned_colnames

['FILE_NAME',
 'FILE_SPECIFICATION_NAME',
 'VOLUME_ID',
 'ANTIBLOOMING_STATE_FLAG',
 'BIAS_STRIP_MEAN',
 'CALIBRATION_LAMP_STATE_FLAG',
 'COMMAND_FILE_NAME',
 'COMMAND_SEQUENCE_NUMBER',
 'DARK_STRIP_MEAN',
 'DATA_CONVERSION_TYPE',
 'DATA_SET_ID',
 'DELAYED_READOUT_FLAG',
 'DESCRIPTION',
 'DETECTOR_TEMPERATURE',
 'EARTH_RECEIVED_START_TIME',
 'EARTH_RECEIVED_STOP_TIME',
 'ELECTRONICS_BIAS',
 'EXPECTED_MAXIMUM_0',
 'EXPECTED_PACKETS',
 'EXPOSURE_DURATION',
 'FILTER_NAME_0',
 'FILTER_TEMPERATURE',
 'FLIGHT_SOFTWARE_VERSION_ID',
 'GAIN_MODE_ID',
 'IMAGE_MID_TIME',
 'IMAGE_NUMBER',
 'IMAGE_OBSERVATION_TYPE',
 'IMAGE_TIME',
 'INSTRUMENT_DATA_RATE',
 'INSTRUMENT_HOST_NAME',
 'INSTRUMENT_ID',
 'INSTRUMENT_MODE_ID',
 'INSTRUMENT_NAME',
 'INST_CMPRS_PARAM_0',
 'INST_CMPRS_PARAM_2',
 'INST_CMPRS_PARAM_3',
 'INST_CMPRS_RATE_0',
 'INST_CMPRS_RATIO',
 'INST_CMPRS_TYPE',
 'LIGHT_FLOOD_STATE_FLAG',
 'METHOD_DESC',
 'MISSING_LINES',
 'MISSING_PACKET_FLAG',
 'MISSION_NAME',
 'MISSION_PHASE_NAME',
 'OBSE

In [133]:
for i, j in zip_longest(colnames, cleaned_colnames):
    print(i)
    print(j)
    print()

FILE_NAME
FILE_NAME

FILE_SPECIFICATION_NAME
FILE_SPECIFICATION_NAME

VOLUME_ID
VOLUME_ID

ANTIBLOOMING_STATE_FLAG
ANTIBLOOMING_STATE_FLAG

BIAS_STRIP_MEAN
BIAS_STRIP_MEAN

CALIBRATION_LAMP_STATE_FLAG
CALIBRATION_LAMP_STATE_FLAG

COMMAND_FILE_NAME
COMMAND_FILE_NAME

COMMAND_SEQUENCE_NUMBER
COMMAND_SEQUENCE_NUMBER

DARK_STRIP_MEAN
DARK_STRIP_MEAN

DATA_CONVERSION_TYPE
DATA_CONVERSION_TYPE

DATA_SET_ID
DATA_SET_ID

DELAYED_READOUT_FLAG
DELAYED_READOUT_FLAG

DESCRIPTION
DESCRIPTION

DETECTOR_TEMPERATURE
DETECTOR_TEMPERATURE

EARTH_RECEIVED_START_TIME
EARTH_RECEIVED_START_TIME

EARTH_RECEIVED_STOP_TIME
EARTH_RECEIVED_STOP_TIME

ELECTRONICS_BIAS
ELECTRONICS_BIAS

EXPECTED_MAXIMUM
EXPECTED_MAXIMUM_0

EXPECTED_PACKETS
EXPECTED_PACKETS

EXPOSURE_DURATION
EXPOSURE_DURATION

FILTER_NAME
FILTER_NAME_0

FILTER_TEMPERATURE
FILTER_TEMPERATURE

FLIGHT_SOFTWARE_VERSION_ID
FLIGHT_SOFTWARE_VERSION_ID

GAIN_MODE_ID
GAIN_MODE_ID

IMAGE_MID_TIME
IMAGE_MID_TIME

IMAGE_NUMBER
IMAGE_NUMBER

IMAGE_OBSERVATION_

In [13]:
def map_string_to_bool(x):
    if x.strip() == 'YES':
        return True
    else:
        return False

In [14]:
df.RINGS_FLAG = df.RINGS_FLAG.map(map_string_to_bool)

In [15]:
rings = df[df.RINGS_FLAG]

In [16]:
rings.TARGET_NAME.value_counts()

SATURN                      2935
SKY                           43
EPIMETHEUS                     7
ATLAS                          7
ENCELADUS                      7
JANUS                          6
PANDORA                        6
PROMETHEUS                     5
DIONE                          3
CALYPSO                        2
HELENE                         1
dtype: int64

In [86]:
s = str(res.read())

In [87]:
from html.parser import HTMLParser

In [93]:
class MyHTMLParser(HTMLParser):
    pdfs = []
    def handle_data(self, data):
        if data.endswith('.pdf'):
            self.pdfs.append(data)
parser = MyHTMLParser()
parser.feed(s)

In [103]:
from urllib.request import urlretrieve
import sys
def report(blocknr, blocksize, size):
    current = blocknr*blocksize
    sys.stdout.write("\r{0:.2f}%".format(100.0*current/size))

def downloadFile(url):
    print("\n",url)
    fname = url.split('/')[-1]
    print(fname)
    urlretrieve(url, fname, report)

In [106]:
for url in parser.pdfs:
    dlurl = "http://lasp.colorado.edu/media/projects/workshops/2014planetaryrings/Presentations/"
    downloadFile(dlurl + url)


 http://lasp.colorado.edu/media/projects/workshops/2014planetaryrings/Presentations/Albers-NewMoonInducedStructure.pdf
Albers-NewMoonInducedStructure.pdf
100.13%
 http://lasp.colorado.edu/media/projects/workshops/2014planetaryrings/Presentations/Becker-MeasuringSubCentimeterParticles.pdf
Becker-MeasuringSubCentimeterParticles.pdf
100.01%
 http://lasp.colorado.edu/media/projects/workshops/2014planetaryrings/Presentations/Bloom-HousekeepingAnnouncement.pdf
Bloom-HousekeepingAnnouncement.pdf
100.41%
 http://lasp.colorado.edu/media/projects/workshops/2014planetaryrings/Presentations/Bradley-UVISReflectanceSpectraComparison.pdf
Bradley-UVISReflectanceSpectraComparison.pdf
100.33%
 http://lasp.colorado.edu/media/projects/workshops/2014planetaryrings/Presentations/Brooks-TowardsUnderstandingThermalThroughput.pdf
Brooks-TowardsUnderstandingThermalThroughput.pdf
100.03%
 http://lasp.colorado.edu/media/projects/workshops/2014planetaryrings/Presentations/Colwell-Studying_Sizes_and_Shapes.pdf
Col

# Using JSON

In [1]:
from pyciss import io, opusapi

In [51]:
opus = opusapi.OPUS()

In [52]:
opus.get_between_resolutions(res1=0.5, res2=0.6)

In [55]:
opus.show_images('med')

In [56]:
opus.get_filename('N1554730872')

[Raw:
 Label:
 http://pds-rings.seti.org/volumes/COISS_2xxx/COISS_2031/data/1554726575_1554802203/N1554730872_1.LBL
 Image:
 http://pds-rings.seti.org/volumes/COISS_2xxx/COISS_2031/data/1554726575_1554802203/N1554730872_1.IMG
 Calibrated:
 Label:
 http://pds-rings.seti.org/derived/COISS_2xxx/COISS_2031/data/1554726575_1554802203/N1554730872_1_CALIB.LBL
 Image:
 http://pds-rings.seti.org/derived/COISS_2xxx/COISS_2031/data/1554726575_1554802203/N1554730872_1_CALIB.IMG]

In [48]:
opus.show_images('med')

In [49]:
opus.download_previews()

In [50]:
opus.download_results()